In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
train=pd.read_csv('train_V2.csv')

In [6]:
test=pd.read_csv('test_V2.csv')

In [9]:
sample = pd.read_csv('sample_submission_V2.csv')
sample.head()

,Id,winPlacePerc
0,9329eb41e215eb,1
1,639bd0dcd7bda8,1
2,63d5c8ef8dfe91,1
3,cf5b81422591d1,1
4,ee6a295187ba21,1


In [26]:
train.head()

,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,kills,killStreaks,...,normal-squad-fpp,crashfpp,normal-duo-fpp,flaretpp,normal-solo-fpp,flarefpp,normal-squad,crashtpp,normal-solo,normal-duo
Id,,,,,,,,,,,,,,,,,,,,,
7f96b2f878858a,0,0,0.00,0,0,0,60,1241,0,0,...,0,0,0,0,0,0,0,0,0,0
eef90569b9d03c,0,0,91.47,0,0,0,57,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1eaf90ac73de72,1,0,68.00,0,0,0,47,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4616d365dd2853,0,0,32.90,0,0,0,75,0,0,0,...,0,0,0,0,0,0,0,0,0,0
315c96c26c9aac,0,0,100.00,0,0,0,45,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [8]:
train.columns

Index(['Id', 'groupId', 'matchId', 'assists', 'boosts', 'damageDealt', 'DBNOs',
       'headshotKills', 'heals', 'killPlace', 'killPoints', 'kills',
       'killStreaks', 'longestKill', 'matchDuration', 'matchType', 'maxPlace',
       'numGroups', 'rankPoints', 'revives', 'rideDistance', 'roadKills',
       'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance',
       'weaponsAcquired', 'winPoints', 'winPlacePerc'],
      dtype='object')

In [12]:
train.drop(columns=['groupId', 'matchId'], inplace=True, errors='ignore')
test.drop(columns=['groupId', 'matchId'], inplace=True, errors='ignore')

In [14]:
train.set_index('Id', inplace=True)
test.set_index('Id', inplace=True)

In [16]:
train.fillna(0, inplace=True)
test.fillna(0, inplace=True)

In [17]:
train[['longestKill', 'matchDuration', 'matchType', 'maxPlace',
       'numGroups', 'rankPoints']].head()

,longestKill,matchDuration,matchType,maxPlace,numGroups,rankPoints
Id,,,,,,
7f96b2f878858a,0.00,1306,squad-fpp,28,26,-1
eef90569b9d03c,0.00,1777,squad-fpp,26,25,1484
1eaf90ac73de72,0.00,1318,duo,50,47,1491
4616d365dd2853,0.00,1436,squad-fpp,31,30,1408
315c96c26c9aac,58.53,1424,solo-fpp,97,95,1560


In [24]:
train['matchType'].value_counts().keys()

Index(['squad-fpp', 'duo-fpp', 'squad', 'solo-fpp', 'duo', 'solo',
       'normal-squad-fpp', 'crashfpp', 'normal-duo-fpp', 'flaretpp',
       'normal-solo-fpp', 'flarefpp', 'normal-squad', 'crashtpp',
       'normal-solo', 'normal-duo'],
      dtype='object')

In [25]:
for x in train['matchType'].value_counts().keys():
    train[x]=train['matchType'].apply(lambda d: 1 if d==x else 0)
train.drop(columns=['matchType'], inplace=True)

for x in test['matchType'].value_counts().keys():
    test[x]=test['matchType'].apply(lambda d: 1 if d==x else 0)
test.drop(columns=['matchType'], inplace=True)

In [27]:
y_train = train['winPlacePerc']
X_train = train.drop(columns='winPlacePerc')

In [28]:
len(X_train.columns)==len(test.columns)

True

In [42]:
from sklearn.model_selection import cross_val_score

from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor

In [43]:
models = [('forest',RandomForestRegressor()), 
          ('ada',AdaBoostRegressor())]

In [44]:
model_scores = {}

In [45]:
for modelo in models:
    score = abs(cross_val_score(modelo[1], X_train, y_train, cv=2, scoring='neg_mean_squared_error', n_jobs=-1)).mean()
    model_scores[modelo[0]]=score
    print(modelo[0])

KeyboardInterrupt: 

In [ ]:
%matplotlib inline

lst = sorted(model_scores.items())
x, y = zip(*lst)
plt.plot(x,y)
plt.show()

In [46]:
ada = AdaBoostRegressor()

In [48]:
ada.fit(X_train, y_train)

AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',
         n_estimators=50, random_state=None)

In [49]:
pred=ada.predict(test)

In [50]:
sample['winPlacePerc']=pred

In [52]:
sample.set_index('Id', inplace=True, drop=True)

In [53]:
sample.head()

,winPlacePerc
Id,
9329eb41e215eb,0.276032
639bd0dcd7bda8,0.757664
63d5c8ef8dfe91,0.556097
cf5b81422591d1,0.644564
ee6a295187ba21,0.840465


In [54]:
sample.to_csv('sub1.csv')